#This project covers following skills:

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [ ]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7fcfe29e4550>)

In [ ]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [ ]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [ ]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
1596,0.4,821.50,-200,1.951846,580.50,34.0,1325.75,47.0,1256.25,602.25,14.175,49.900000,0.802525
3300,-200.0,1022.50,-200,6.776210,847.00,120.0,829.50,102.0,1662.75,916.25,25.400,48.499999,1.550735
4820,1.6,961.00,-200,9.241702,948.75,230.0,853.00,99.0,1305.50,812.25,23.800,26.975000,0.784509
8067,2.9,-200.00,-200,-200.000000,-200.00,254.3,-200.00,186.6,-200.00,-200.00,-200.000,-200.000000,-200.000000
1856,-200.0,845.75,-200,3.581202,687.00,-200.0,1294.75,-200.0,1288.25,725.75,18.600,38.775001,0.822812


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [ ]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [ ]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [ ]:
np.random.seed(70629)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [ ]:
#In this task the following datasets wil be used:
#data_training_unproc - Select the training data
#data_val_unproc - Select the val data
#data_test_unproc - Select the test data

In [ ]:
#Functions below are created for the first and second point of the Question 1.a
#Missing values(-200) replacement with mean
def replace_miss(dataset, mean_set=None):
  column_means = {}
  try:
    if mean_set!= None:
      column_means = mean_set
    for column in dataset.columns:
      #temp_col is a temporary array of all the elements from the selected columns exclduing the -200 value
      if mean_set!= None:
        dataset[column] = [value if value != -200 else mean_set[column] for value in dataset[column]]
      else:
        temp_col = [i for i in dataset[column] if i != -200]
        mean_col = np.mean(temp_col)
        column_means[column] = mean_col
      #final -200 values replacement
        dataset[column] = [value if value != -200 else mean_col for value in dataset[column]]
    return column_means
  except:
    return "sorry, error occured during this replacement"

#Additional separate function for mean calculation
def mean_calc(single_column):
  meancol = np.mean(single_column)
  return meancol


def standarization(dataset, mean_set=None, std_set=None):
    column_std = {}
    try:
      if std_set != None:
        column_std = std_set
      for column in dataset.columns:
        #standarizatin function is also prepared for handling datasets not 
        #being preprocessed with replace_miss function earlier
        if mean_set != None:
          mean_col = mean_set[column]
        else:
          temp_col = [i for i in dataset[column] if i != -200]
          mean_col = np.mean(temp_col)
        if std_set != None:
          stand_dev = std_set[column]
        else:
          stand_dev = np.std(dataset[column])
          column_std[column] = stand_dev
        dataset[column] = [(value - mean_col) / stand_dev for value in dataset[column]]
      return column_std
    except:
      return "sorry, error occured during this standarization"

In [ ]:
#for better data analysis and understanding the dataframe of the training set is displayed below
data_training_unproc.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6656,1.8,994.50,5.902703,807.25,213.0,977.75,90.0,1055.50,1043.50,6.700000,64.650000,0.638121
2723,0.5,886.50,2.916438,646.75,50.0,1046.25,61.0,1500.25,740.25,25.150000,47.974999,1.511502
7873,4.1,1258.75,17.044564,1211.25,502.1,559.00,187.6,1243.00,1530.75,9.375000,36.675000,0.432383
2607,1.6,1062.50,7.868564,893.75,104.0,822.50,100.0,1688.00,1025.25,27.199999,41.125000,1.461106
9333,1.1,956.00,5.395234,783.00,141.9,856.75,100.4,895.75,515.75,30.000000,11.075000,0.462436
2652,0.7,955.75,5.410637,783.75,57.0,927.50,62.0,1633.75,933.75,25.275000,47.599999,1.510786
562,0.8,986.25,2.987862,651.25,60.0,1144.50,74.0,1380.25,891.00,14.450000,57.975001,0.948831
5460,1.2,1029.25,7.528240,879.50,182.0,708.50,47.0,1450.75,964.75,17.275000,78.375000,1.532012
6821,2.9,1182.50,9.125831,944.25,382.0,731.00,126.0,1226.25,1091.75,6.625000,83.325001,0.818323
2265,0.6,905.50,4.146091,718.75,-200.0,1050.25,-200.0,1560.50,612.25,24.350000,48.174999,1.447627


In [ ]:
#First and second point - (missing values + standarization) preprocessing the training data
training_set = data_training_unproc.copy()

#the variable miss_vals stores means for all the columns
miss_vals = replace_miss(training_set)

#the variable standard_vals stores standard deviations for all the columns
standard_vals = standarization(training_set, mean_set=miss_vals)

In [ ]:
#as we can see missing values are replaced and standarization is applied
training_set.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6656,-0.246034,-0.544423,-0.602387,-0.539725,-0.202993,0.593174,-0.522034,-1.145352,-0.003023,-1.281566,0.904818,-0.902549
2723,-1.135509,-1.049518,-1.010901,-1.155658,-0.972666,0.863797,-1.135805,0.156912,-0.766512,0.860155,-0.069879,1.342413
7873,1.327654,0.691423,0.921792,1.010663,1.162115,-1.061181,1.543624,-0.596337,1.223721,-0.971046,-0.730393,-1.431382
2607,-0.382876,-0.226400,-0.333463,-0.207773,-0.717682,-0.020172,-0.310388,0.706660,-0.048971,1.098123,-0.470279,1.212873
9333,-0.724982,-0.724480,-0.671808,-0.632786,-0.538721,0.115139,-0.301923,-1.613114,-1.331734,1.423154,-2.226779,-1.354134
2652,-0.998667,-0.725649,-0.669701,-0.629908,-0.939613,0.394651,-1.114640,0.547811,-0.279339,0.874665,-0.091799,1.340573
562,-0.930245,-0.583006,-1.001130,-1.138389,-0.925447,1.251953,-0.860666,-0.194458,-0.386971,-0.381927,0.514647,-0.103891
5460,-0.656561,-0.381904,-0.380018,-0.262459,-0.349372,-0.470552,-1.432108,0.011972,-0.201291,-0.053995,1.707080,1.395131
6821,0.506600,0.334817,-0.161472,-0.013974,0.595012,-0.381661,0.239889,-0.645382,0.118456,-1.290272,1.996420,-0.439352
2265,-1.067088,-0.960658,-0.842688,-0.879352,0.000000,0.879599,0.000000,0.333329,-1.088777,0.767289,-0.058188,1.178226


## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [ ]:
# Answer 1.4 - Spliting the preprocessed training set to features and target value
yTrain = training_set.iloc[:,0]
xTrain = training_set.iloc[:,1:]

In [ ]:
#target variable CO(GT) is removed from xTrain
xTrain

,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6656,-0.544423,-0.602387,-0.539725,-0.202993,0.593174,-0.522034,-1.145352,-0.003023,-1.281566,0.904818,-0.902549
2723,-1.049518,-1.010901,-1.155658,-0.972666,0.863797,-1.135805,0.156912,-0.766512,0.860155,-0.069879,1.342413
7873,0.691423,0.921792,1.010663,1.162115,-1.061181,1.543624,-0.596337,1.223721,-0.971046,-0.730393,-1.431382
2607,-0.226400,-0.333463,-0.207773,-0.717682,-0.020172,-0.310388,0.706660,-0.048971,1.098123,-0.470279,1.212873
9333,-0.724480,-0.671808,-0.632786,-0.538721,0.115139,-0.301923,-1.613114,-1.331734,1.423154,-2.226779,-1.354134
...,...,...,...,...,...,...,...,...,...,...,...
6524,0.684408,0.774816,0.885941,0.939713,-0.037950,0.409205,0.287212,0.931040,0.192681,-0.118102,0.158752
136,2.917583,2.017620,1.852055,0.661119,-0.787595,0.747838,2.238047,2.010499,0.009851,-0.367987,-0.313652
5654,-0.207693,-0.217138,-0.075376,-0.202993,-0.459687,-1.347450,0.343578,0.664794,0.154954,1.544874,1.703228
461,-0.987550,-1.020036,-1.171968,-0.949057,2.019376,-0.860666,-1.046530,-1.484683,-0.616994,-1.010965,-1.364720


In [ ]:
#target variable CO(GT) - yTrain
yTrain

6656   -0.246034
2723   -1.135509
7873    1.327654
2607   -0.382876
9333   -0.724982
          ...   
6524    0.985548
136     2.148708
5654   -0.451297
461    -0.793403
806     1.601338
Name: CO(GT), Length: 5372, dtype: float64

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [ ]:
#displaying the unprocessed validation set to analyze the data
data_val_unproc.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8286,2.7,1156.00,8.239851,909.00,456.4,684.25,250.4,1022.50,1384.25,4.125000,54.975000,0.455840
765,0.9,854.00,3.096375,658.00,42.0,1408.50,46.0,1107.75,368.50,22.050000,18.975000,0.496923
2350,2.3,1100.50,12.199629,1056.50,122.0,796.50,113.0,1772.00,862.50,33.000000,26.025000,1.287704
8452,2.7,1206.75,11.249663,1023.25,473.0,629.00,188.0,1233.00,1373.00,6.200000,62.425001,0.595807
5555,0.8,905.00,5.247306,775.75,163.0,850.75,48.0,1327.25,825.00,15.200000,77.974998,1.338227
6009,0.6,736.75,0.925975,491.50,-200.0,1390.75,-200.0,880.25,467.00,11.225000,41.325000,0.549830
1366,3.6,1271.50,14.602582,1136.00,209.0,729.50,139.0,1868.50,1076.50,12.700000,74.225002,1.086300
0,2.6,1360.00,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.600000,48.875001,0.757754
3623,0.9,907.75,4.068437,714.50,38.0,987.75,46.0,1479.00,495.00,34.999999,31.050000,1.717252
2750,2.3,1172.75,14.422632,1130.25,208.0,647.75,120.0,1885.00,1401.00,23.225000,43.925000,1.234377


In [ ]:
#First bulletpoint - replacing missing values in validation set and standarization
validation_set = data_val_unproc.copy()
miss_validation = replace_miss(validation_set, mean_set=miss_vals)
standard_validation = standarization(validation_set, mean_set=miss_vals, std_set=standard_vals)

In [ ]:
validation_set.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8286,0.369757,0.210882,-0.282671,-0.149250,0.946323,-0.566356,2.872756,-1.241979,0.854880,-1.580478,0.339289,-1.371088
765,-0.861824,-1.201514,-0.986286,-1.112485,-1.010442,2.294937,-1.453273,-0.992360,-1.702463,0.500299,-1.765004,-1.265488
2350,0.096072,-0.048681,0.259016,0.416795,-0.632688,-0.122890,-0.035250,0.952619,-0.458725,1.771402,-1.352913,0.767157
8452,0.369757,0.448229,0.129064,0.289195,1.024707,-0.784632,1.552090,-0.625618,0.826556,-1.339607,0.774761,-1.011313
5555,-0.930245,-0.962997,-0.692044,-0.660609,-0.439089,0.091435,-1.410944,-0.349646,-0.553138,-0.294866,1.683698,0.897021
6009,-1.067088,-1.749869,-1.283191,-1.751445,0.000000,2.224812,0.000000,-1.658499,-1.454471,-0.756293,-0.458589,-1.129494
1366,0.985548,0.751053,0.587735,0.721884,-0.221880,-0.387587,0.515028,1.235179,0.080061,-0.585072,1.464501,0.249461
0,0.301336,1.164950,0.215528,0.374582,-0.424923,0.903304,-0.035250,0.718372,0.560940,-0.480597,-0.017272,-0.595041
3623,-0.861824,-0.950135,-0.853310,-0.895661,-1.029329,0.632681,-1.453273,0.094691,-1.383976,2.003566,-1.059189,1.871278
2750,0.096072,0.289218,0.563118,0.699818,-0.226602,-0.710557,0.112902,1.283492,0.897051,0.636696,-0.306612,0.630083


In [ ]:
#second point - grid creation for the regularization parameter, batch size and learning rate(5 values for each parameter)
#in case of the minibatch size value needs to be an integer as it represents the number of datapoints
size_grid = np.linspace(5, 100, 5, dtype=int)
learning_rate_grid = np.logspace(-6,-2,5)
regularization_parameter_grid = np.logspace(-4,1,5)

#creating parameters dataframe
parameters = [size_grid, learning_rate_grid, regularization_parameter_grid]
parameters = np.array(parameters).T

#displaying the parameters values
grid_parameters = pd.DataFrame(parameters, columns=["minibatch size", "learning rate", "regularization parameter"])
grid_parameters

,minibatch size,learning rate,regularization parameter
0,5.0,0.000001,0.000100
1,28.0,0.000010,0.001778
2,52.0,0.000100,0.031623
3,76.0,0.001000,0.562341
4,100.0,0.010000,10.000000


In [ ]:
#Spliting the validation set
yVal = validation_set.iloc[:,0]
xVal = validation_set.iloc[:,1:]

In [ ]:
#third point - For each value that you have of  γ ,  η  and  S  from the previous step, use the training set to compute  w  using minibatch gradient descent and then measure the RMSE over the validation data.
# For the validation data, make sure you preprocess it before applying the prediction model over it. 
#For the minibatch gradient descent choose to stop the iterative procedure after  200  iterations
import random

#seed based on my Ucard number
np.random.seed(70629)
random.seed(70629)

#Best RMSE and parameter values initialization
min_rmse = np.inf
min_batch_size = np.inf
min_eta = np.inf
min_gamma = np.inf

#number of epochs
num_epochs = 200

#initialization of the parameters list
rmse_parameter = []


def gradient_calculation(xSet, ySet, weights, regularization=0, lr=0.001, batch_size=10):
  r = random.sample(range(xSet.shape[0]), batch_size)
  xBatch = xSet.iloc[r]
  yBatch = ((ySet.iloc[r]).values).reshape(-1,1)

  gradient = (-np.dot(xBatch.T, yBatch) + np.dot(np.dot(xBatch.T,xBatch),weights)) + np.dot(regularization,weights)
  weights = weights - lr * gradient
  return weights

def RMSE_calculation(xValSet, yValSet, weights):
  valWeights = (np.dot(xValSet, weights))
  diff_mse = np.squeeze(np.asarray((yValSet.values).reshape(-1,1) - valWeights))
  mse = ((1/len(xValSet))*np.dot(diff_mse, diff_mse.T))
  return np.sqrt(mse)

#minibatch gradient descent
for gamma in regularization_parameter_grid:   
    for eta in learning_rate_grid:        
        for batch_size in size_grid:    

            #random weights initialization
            weights = 0.5*np.random.randn(11,1)
            for i in range(num_epochs):
                weights = gradient_calculation(xSet = xTrain,ySet = yTrain, regularization = gamma, lr = eta, batch_size = batch_size, weights= weights)

            rmse = RMSE_calculation(xVal, yVal, weights)
            
            rmse_parameter.append([gamma, eta, batch_size, rmse])
            if rmse < min_rmse:
                min_rmse = rmse
                min_batch_size = batch_size
                min_eta = eta
                min_gamma = gamma
            print('Parameters: α: {}, η: {}, β: {}, result in RMSE: {}'.format(gamma, eta, batch_size, rmse))

print('Lowest RMSE recorded:', min_rmse)


Parameters: α: 0.0001, η: 1e-06, β: 5, result in RMSE: 1.9733832775089881
Parameters: α: 0.0001, η: 1e-06, β: 28, result in RMSE: 2.78463104876181
Parameters: α: 0.0001, η: 1e-06, β: 52, result in RMSE: 1.5951887579987856
Parameters: α: 0.0001, η: 1e-06, β: 76, result in RMSE: 1.517301598943861
Parameters: α: 0.0001, η: 1e-06, β: 100, result in RMSE: 2.0313748015945507
Parameters: α: 0.0001, η: 1e-05, β: 5, result in RMSE: 3.1140690770616164
Parameters: α: 0.0001, η: 1e-05, β: 28, result in RMSE: 1.689377706972772
Parameters: α: 0.0001, η: 1e-05, β: 52, result in RMSE: 0.9657505308991698
Parameters: α: 0.0001, η: 1e-05, β: 76, result in RMSE: 1.3196425118946369
Parameters: α: 0.0001, η: 1e-05, β: 100, result in RMSE: 0.982870887583259
Parameters: α: 0.0001, η: 0.0001, β: 5, result in RMSE: 1.4340705438333425
Parameters: α: 0.0001, η: 0.0001, β: 28, result in RMSE: 0.5210522089644318
Parameters: α: 0.0001, η: 0.0001, β: 52, result in RMSE: 0.47480280400945823
Parameters: α: 0.0001, η: 0

In [ ]:
#forth point answer - choosing the parameters resulting in the lowest RMSE
rmse_parameter_frame = pd.DataFrame(rmse_parameter,columns=["regularization", "learning rate", "batch size", "RMSE"])
rmse_parameter_frame

,regularization,learning rate,batch size,RMSE
0,0.0001,0.000001,5,1.973383e+00
1,0.0001,0.000001,28,2.784631e+00
2,0.0001,0.000001,52,1.595189e+00
3,0.0001,0.000001,76,1.517302e+00
4,0.0001,0.000001,100,2.031375e+00
...,...,...,...,...
120,10.0000,0.010000,5,4.190027e-01
121,10.0000,0.010000,28,5.679314e-01
122,10.0000,0.010000,52,5.271085e+64
123,10.0000,0.010000,76,3.463865e+110


In [ ]:
#forth point answer
print('best batch size: ', min_batch_size)
print('best learning rate: ', min_eta)
print('best L2 regularization: ', min_gamma)
print('minimum RMSE: ', min_rmse)

best batch size:  100
best learning rate:  0.001
best L2 regularization:  0.5623413251903491
minimum RMSE:  0.33836392234256946


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [ ]:
# first point join unprocesssed training and validation sets and process them

#my preprocessing methods require keeping the same names of the columns, threfore pd.concat was used instead of np.vstack 
new_training_set = [data_training_unproc.copy(), data_val_unproc.copy()]
new_training_set = pd.concat(new_training_set)

#first point - impute missing values with mean
miss_new_train = replace_miss(new_training_set)

#second point - standarization
standard_new_train = standarization(new_training_set, mean_set=miss_new_train)

In [ ]:
# new training set displayed for analysis and proof it is preprocessed
new_training_set[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6656,-0.245369,-0.541535,-0.603990,-0.539757,-0.205147,0.597626,-0.526989,-1.136637,-0.000027,-1.277596,0.909522,-0.897902
2723,-1.138606,-1.046059,-1.014735,-1.156916,-0.979335,0.869747,-1.144603,0.161757,-0.763948,0.848368,-0.060787,1.340208
7873,1.334972,0.692912,0.928514,1.013714,1.167969,-1.065888,1.551602,-0.589253,1.227410,-0.969360,-0.718329,-1.425120
2607,-0.382790,-0.223872,-0.333596,-0.207145,-0.722856,-0.019116,-0.314018,0.709870,-0.046001,1.084586,-0.459385,1.211064
9333,-0.726343,-0.721389,-0.673790,-0.633004,-0.542845,0.116945,-0.305500,-1.603007,-1.329488,1.407225,-2.207979,-1.348108
2652,-1.001185,-0.722557,-0.671671,-0.630120,-0.946088,0.398004,-1.123306,0.551493,-0.276500,0.862771,-0.082609,1.338374
562,-0.932474,-0.580075,-1.004911,-1.139612,-0.931839,1.260052,-0.867741,-0.188569,-0.384192,-0.384576,0.521107,-0.101681
5460,-0.657632,-0.379200,-0.380406,-0.261939,-0.352385,-0.471989,-1.442761,0.017247,-0.198407,-0.059056,1.708173,1.392766
6821,0.510446,0.336709,-0.160666,-0.012961,0.597539,-0.382606,0.239704,-0.638153,0.121520,-1.286238,1.996211,-0.436118
2265,-1.069895,-0.957300,-0.845603,-0.880059,0.000000,0.885637,0.000000,0.337649,-1.086394,0.756185,-0.049150,1.176523


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [ ]:
# first point - missing data removal and standarization
test_set = data_test_unproc.copy()
miss_test = replace_miss(test_set, mean_set=miss_new_train)
standard_test = standarization(test_set, mean_set=miss_new_train, std_set=standard_new_train)

In [ ]:
# test set displayed for analysis and proof it is preprocessed
test_set[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6607,-0.382790,-1.190876,-0.807220,-0.822381,0.032334,0.847898,0.133219,-1.728540,-0.660664,-0.612152,-1.455880,-1.645976
8128,0.373025,1.265173,0.216417,0.374446,1.138996,-1.137394,2.271441,-0.403873,1.668254,-1.156606,0.519653,-0.960859
5775,0.510446,0.474519,0.833690,0.933926,0.944261,-1.096675,-0.377909,0.412093,0.419405,-0.214614,1.206289,0.677807
4664,-1.069895,-1.251606,-0.995540,-1.123270,-0.955587,0.852864,-1.378870,-0.508970,-0.954141,0.499802,0.043954,0.822370
530,1.884657,1.986922,1.363894,1.363630,0.060832,-0.640823,0.985101,1.392274,1.398709,0.053292,-0.472478,-0.356630
477,-0.176659,0.055251,-0.321964,-0.193686,-0.656361,0.650262,-0.505692,-0.213383,-0.534709,0.148356,-1.310407,-1.049327
4012,-0.382790,-0.191172,-0.467449,-0.366722,-0.756103,-0.488872,-0.633474,0.565360,-0.444021,1.185411,-0.060787,2.062278
6086,1.815946,1.717142,1.403673,1.394392,1.789694,-1.333043,1.687903,0.963126,1.255120,-0.352888,0.820783,0.160893
6580,-0.932474,-1.486349,-1.143739,-1.403971,-0.718106,4.547354,-1.038117,-1.455579,-1.325709,-0.635197,-0.020055,-0.756894
2789,-0.176659,-0.114092,-0.059199,0.095667,-0.642112,-0.244559,-0.228830,0.616450,0.115222,0.865652,-0.572855,0.657833


In [ ]:
#before moving to the second point both new train and test sets need to be splitted

yNewtrain = new_training_set.iloc[:,0]
xNewtrain = new_training_set.iloc[:,1:]

yTest = test_set.iloc[:,0]
xTest = test_set.iloc[:,1:]

In [ ]:
#second bulletpoint - Use the best values of  γ ,  η  and  S  found in the validation set and train a new regularised linear model with stochastic gradient descent

#basic initialization
np.random.seed(70629)
num_epochs = 200
new_weights = 0.5*np.random.randn(11,1)
#parameters are taken from the min variables previous task(min_eta, min_gamma, min_batch_size)

for i in range(num_epochs):
    new_weights = gradient_calculation(xSet = xNewtrain,ySet = yNewtrain, regularization = min_gamma, lr = min_eta, batch_size = min_batch_size, weights= new_weights)

rmse = RMSE_calculation(xTest, yTest, new_weights)


#third point -  RMSE on the test set
print('The RMSE on the test set is: {}'.format( rmse))


The RMSE on the test set is: 0.3369997960379572


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [ ]:
##new initialized sets for the Question 2 
training_setQ2 = data_training_unproc.copy()
validation_setQ2 = data_val_unproc.copy()
test_setQ2 = data_test_unproc.copy()

In [ ]:
#displaying and checking the training set for Q2 
training_setQ2.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6656,1.8,994.50,5.902703,807.25,213.0,977.75,90.0,1055.50,1043.50,6.700000,64.650000,0.638121
2723,0.5,886.50,2.916438,646.75,50.0,1046.25,61.0,1500.25,740.25,25.150000,47.974999,1.511502
7873,4.1,1258.75,17.044564,1211.25,502.1,559.00,187.6,1243.00,1530.75,9.375000,36.675000,0.432383
2607,1.6,1062.50,7.868564,893.75,104.0,822.50,100.0,1688.00,1025.25,27.199999,41.125000,1.461106
9333,1.1,956.00,5.395234,783.00,141.9,856.75,100.4,895.75,515.75,30.000000,11.075000,0.462436
2652,0.7,955.75,5.410637,783.75,57.0,927.50,62.0,1633.75,933.75,25.275000,47.599999,1.510786
562,0.8,986.25,2.987862,651.25,60.0,1144.50,74.0,1380.25,891.00,14.450000,57.975001,0.948831
5460,1.2,1029.25,7.528240,879.50,182.0,708.50,47.0,1450.75,964.75,17.275000,78.375000,1.532012
6821,2.9,1182.50,9.125831,944.25,382.0,731.00,126.0,1226.25,1091.75,6.625000,83.325001,0.818323
2265,0.6,905.50,4.146091,718.75,-200.0,1050.25,-200.0,1560.50,612.25,24.350000,48.174999,1.447627


In [ ]:
#first point Simple Imputer on training set
from sklearn.impute import SimpleImputer

Q2_imputer = SimpleImputer(missing_values=-200, strategy='mean')
Q2_imputer.fit(training_setQ2)
training_setQ2 = Q2_imputer.transform(training_setQ2)

In [ ]:
#Second point - Standard Scaler on the training set
from sklearn.preprocessing import StandardScaler

Q2_scaler = StandardScaler()
Q2_scaler.fit(training_setQ2)
training_setQ2 = Q2_scaler.transform(training_setQ2)


In [ ]:
#training set display after preprocessing
pd.DataFrame(training_setQ2[:20])

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.246034,-0.544423,-0.602387,-0.539725,-2.029927e-01,0.593174,-5.220337e-01,-1.145352,-0.003023,-1.281566,0.904818,-0.902549
1,-1.135509,-1.049518,-1.010901,-1.155658,-9.726662e-01,0.863797,-1.135805e+00,0.156912,-0.766512,0.860155,-0.069879,1.342413
2,1.327654,0.691423,0.921792,1.010663,1.162115e+00,-1.061181,1.543624e+00,-0.596337,1.223721,-0.971046,-0.730393,-1.431382
3,-0.382876,-0.226400,-0.333463,-0.207773,-7.176823e-01,-0.020172,-3.103885e-01,0.706660,-0.048971,1.098123,-0.470279,1.212873
4,-0.724982,-0.724480,-0.671808,-0.632786,-5.387214e-01,0.115139,-3.019227e-01,-1.613114,-1.331734,1.423154,-2.226779,-1.354134
5,-0.998667,-0.725649,-0.669701,-0.629908,-9.396127e-01,0.394651,-1.114640e+00,0.547811,-0.279339,0.874665,-0.091799,1.340573
6,-0.930245,-0.583006,-1.001130,-1.138389,-9.254469e-01,1.251953,-8.606661e-01,-0.194458,-0.386971,-0.381927,0.514647,-0.103891
7,-0.656561,-0.381904,-0.380018,-0.262459,-3.493723e-01,-0.470552,-1.432108e+00,0.011972,-0.201291,-0.053995,1.707080,1.395131
8,0.506600,0.334817,-0.161472,-0.013974,5.950124e-01,-0.381661,2.398892e-01,-0.645382,0.118456,-1.290272,1.996420,-0.439352
9,-1.067088,-0.960658,-0.842688,-0.879352,2.684103e-16,0.879599,6.015320e-16,0.333329,-1.088777,0.767289,-0.058188,1.178226


In [ ]:
#third point - Pipeline creation including the simple imputer and standard scaler
from sklearn.pipeline import Pipeline
pipelineQ2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean', missing_values=-200)),
    ('scaler', StandardScaler())
])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [ ]:
#display the validation set before applying the preprocessing pipeline
validation_setQ2.iloc[:20]

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8286,2.7,1156.00,8.239851,909.00,456.4,684.25,250.4,1022.50,1384.25,4.125000,54.975000,0.455840
765,0.9,854.00,3.096375,658.00,42.0,1408.50,46.0,1107.75,368.50,22.050000,18.975000,0.496923
2350,2.3,1100.50,12.199629,1056.50,122.0,796.50,113.0,1772.00,862.50,33.000000,26.025000,1.287704
8452,2.7,1206.75,11.249663,1023.25,473.0,629.00,188.0,1233.00,1373.00,6.200000,62.425001,0.595807
5555,0.8,905.00,5.247306,775.75,163.0,850.75,48.0,1327.25,825.00,15.200000,77.974998,1.338227
6009,0.6,736.75,0.925975,491.50,-200.0,1390.75,-200.0,880.25,467.00,11.225000,41.325000,0.549830
1366,3.6,1271.50,14.602582,1136.00,209.0,729.50,139.0,1868.50,1076.50,12.700000,74.225002,1.086300
0,2.6,1360.00,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.600000,48.875001,0.757754
3623,0.9,907.75,4.068437,714.50,38.0,987.75,46.0,1479.00,495.00,34.999999,31.050000,1.717252
2750,2.3,1172.75,14.422632,1130.25,208.0,647.75,120.0,1885.00,1401.00,23.225000,43.925000,1.234377


In [ ]:
#ANSWER to 2.b
training_setQ2_pipe = data_training_unproc.copy()
pipelineQ2.fit(training_setQ2_pipe)
validation_setQ2 = pipelineQ2.transform(validation_setQ2)

In [ ]:
#validation set display after preprocessing
pd.DataFrame(validation_setQ2[:20])

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.369757,0.210882,-0.282671,-0.149250,9.463235e-01,-0.566356,2.872756e+00,-1.241979,0.854880,-1.580478,0.339289,-1.371088
1,-0.861824,-1.201514,-0.986286,-1.112485,-1.010442e+00,2.294937,-1.453273e+00,-0.992360,-1.702463,0.500299,-1.765004,-1.265488
2,0.096072,-0.048681,0.259016,0.416795,-6.326877e-01,-0.122890,-3.524965e-02,0.952619,-0.458725,1.771402,-1.352913,0.767157
3,0.369757,0.448229,0.129064,0.289195,1.024707e+00,-0.784632,1.552090e+00,-0.625618,0.826556,-1.339607,0.774761,-1.011313
4,-0.930245,-0.962997,-0.692044,-0.660609,-4.390888e-01,0.091435,-1.410944e+00,-0.349646,-0.553138,-0.294866,1.683698,0.897021
5,-1.067088,-1.749869,-1.283191,-1.751445,2.684103e-16,2.224812,6.015320e-16,-1.658499,-1.454471,-0.756293,-0.458589,-1.129494
6,0.985548,0.751053,0.587735,0.721884,-2.218804e-01,-0.387587,5.150280e-01,1.235179,0.080061,-0.585072,1.464501,0.249461
7,0.301336,1.164950,0.215528,0.374582,-4.249231e-01,0.903304,-3.524965e-02,0.718372,0.560940,-0.480597,-0.017272,-0.595041
8,-0.861824,-0.950135,-0.853310,-0.895661,-1.029329e+00,0.632681,-1.453273e+00,0.094691,-1.383976,2.003566,-1.059189,1.871278
9,0.096072,0.289218,0.563118,0.699818,-2.266023e-01,-0.710557,1.129020e-01,1.283492,0.897051,0.636696,-0.306612,0.630083


## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [ ]:
#joining training and validation sets
new_training_setQ2 = np.vstack((training_setQ2, validation_setQ2))
new_training_setQ2.shape

(6523, 12)

In [ ]:
#first point - predefinedSplit - Use PredefinedSplit to tell the cross validator which instances to use for training and which ones for validation
from sklearn.model_selection import PredefinedSplit
test_fold = np.zeros((np.shape(new_training_setQ2)[0], 1))
test_fold[0:np.shape(data_training_unproc)[0]] = 1
ps = PredefinedSplit(test_fold)
print(ps)

PredefinedSplit(test_fold=array([1, 1, ..., 0, 0]))


In [ ]:
# second point - Create a grid of values to explore that include a range of at least five values for each parameter n_estimators, max_features and max_samples

n_estimators_grid = np.linspace(10,400, 5, dtype=int)
max_features_grid = np.linspace(2,11,5, dtype=int)
max_samples_grid = np.linspace(5, 300, 5, dtype=int)

#creating random forest parameters dataframe
forest_parameters = [n_estimators_grid, max_features_grid, max_samples_grid]
forest_parameters = np.array(forest_parameters).T

#displaying the forest parameters values
grid_forest_parameters = pd.DataFrame(forest_parameters, columns=["n_estimators_grid", "max_features_grid", "max_samples_grid"])
grid_forest_parameters

,n_estimators_grid,max_features_grid,max_samples_grid
0,10,2,5
1,107,4,78
2,205,6,152
3,302,8,226
4,400,11,300


In [ ]:
yNewTrainSetQ2 = new_training_setQ2[:,0]
xNewTrainSetQ2 = new_training_setQ2[:,1:]

In [ ]:
pd.DataFrame(xNewTrainSetQ2)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.544423,-0.602387,-0.539725,-2.029927e-01,0.593174,-5.220337e-01,-1.145352,-0.003023,-1.281566,0.904818,-0.902549
1,-1.049518,-1.010901,-1.155658,-9.726662e-01,0.863797,-1.135805e+00,0.156912,-0.766512,0.860155,-0.069879,1.342413
2,0.691423,0.921792,1.010663,1.162115e+00,-1.061181,1.543624e+00,-0.596337,1.223721,-0.971046,-0.730393,-1.431382
3,-0.226400,-0.333463,-0.207773,-7.176823e-01,-0.020172,-3.103885e-01,0.706660,-0.048971,1.098123,-0.470279,1.212873
4,-0.724480,-0.671808,-0.632786,-5.387214e-01,0.115139,-3.019227e-01,-1.613114,-1.331734,1.423154,-2.226779,-1.354134
...,...,...,...,...,...,...,...,...,...,...,...
6518,-0.574822,-0.584802,-0.516699,-9.632223e-01,0.393664,-1.135805e+00,0.435812,-1.129689,1.823639,-1.247698,1.086307
6519,-0.262645,-0.239443,-0.100320,-7.601796e-01,-0.171286,-7.548435e-01,0.549275,-0.137090,1.434763,-0.889676,1.121868
6520,3.571166,3.206619,2.642599,4.486327e-01,-1.365385,9.171540e-01,2.925414,2.621668,-0.042386,-0.083031,-0.128632
6521,-0.263814,-0.003771,0.153920,2.684103e-16,-0.427094,6.015320e-16,0.126167,-0.429771,1.434763,-1.196552,0.555168


In [ ]:
# third point - Train a random forest for regression model that uses the grid of parameters you created before. Use GridSearchCV to find the best set of parameters by performing cross-validation over the predefined split. 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest_parameters_grid = dict(n_estimators = n_estimators_grid, max_samples = max_samples_grid, max_features = max_features_grid)
grid_regression_Question3 = GridSearchCV(RandomForestRegressor(), param_grid=forest_parameters_grid, cv=ps, scoring='neg_mean_squared_error')
grid_regression_Question3.fit(xNewTrainSetQ2, yNewTrainSetQ2)

GridSearchCV(cv=PredefinedSplit(test_fold=array([1, 1, ..., 0, 0])),
             estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 2,  4,  6,  8, 11]),
                         'max_samples': array([  5,  78, 152, 226, 300]),
                         'n_estimators': array([ 10, 107, 205, 302, 400])},
             scoring='neg_mean_squared_error')

In [ ]:
# fourth point - Print the best values in your grid for n_estimators, max_features and max_samples

print('Best number of estimators: ', grid_regression_Question3.best_params_["n_estimators"])
print('Best number of samples: ', grid_regression_Question3.best_params_["max_samples"])
print('Best number of features: ', grid_regression_Question3.best_params_["max_features"])

Best number of estimators:  302
Best number of samples:  300
Best number of features:  11


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [ ]:
# first point - Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data
##new initialized sets for the Question 2.d
training_setQ2 = data_training_unproc.copy()
validation_setQ2 = data_val_unproc.copy()
test_setQ2 = data_test_unproc.copy()

whole_training_setQ2 = np.vstack((training_setQ2, validation_setQ2))

#the pipelineQ2 will be reused for this task
pipelineQ2.fit(whole_training_setQ2)
whole_training_setQ2 = pipelineQ2.transform(whole_training_setQ2)

In [ ]:
#second point - pipeline application on the test set
test_setQ2 = pipelineQ2.transform(test_setQ2)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
pd.DataFrame(test_setQ2)

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.382790,-1.190876,-0.807220,-0.822381,0.032334,0.847898,0.133219,-1.728540,-0.660664,-0.612152,-1.455880,-1.645976
1,0.373025,1.265173,0.216417,0.374446,1.138996,-1.137394,2.271441,-0.403873,1.668254,-1.156606,0.519653,-0.960859
2,0.510446,0.474519,0.833690,0.933926,0.944261,-1.096675,-0.377909,0.412093,0.419405,-0.214614,1.206289,0.677807
3,-1.069895,-1.251606,-0.995540,-1.123270,-0.955587,0.852864,-1.378870,-0.508970,-0.954141,0.499802,0.043954,0.822370
4,1.884657,1.986922,1.363894,1.363630,0.060832,-0.640823,0.985101,1.392274,1.398709,0.053292,-0.472478,-0.356630
...,...,...,...,...,...,...,...,...,...,...,...,...
1146,-0.451501,-0.228544,-0.380406,-0.261939,-0.452127,-0.429283,-1.123306,0.393847,-0.301691,0.594865,0.967712,2.117384
1147,0.304315,-0.488981,-0.004890,0.152384,0.136826,0.796254,0.197110,-0.578306,-0.201556,-0.378814,-0.209171,-0.621969
1148,-0.245369,-0.693360,-0.389338,-0.272514,0.744777,0.054377,0.111922,-1.390622,-0.237453,-0.983763,-0.667413,-1.407981
1149,0.304315,1.589843,1.344095,1.348249,1.512791,-1.491946,0.716758,0.874085,1.486248,-0.430667,0.950255,0.138802


In [ ]:
ywhole_training_setQ2 = whole_training_setQ2[:,0]
xwhole_training_setQ2 = whole_training_setQ2[:,1:]

In [ ]:
ytest_setQ2 = test_setQ2[:,0]
xtest_setQ2 = test_setQ2[:,1:]

In [ ]:
#third point - Fit a random forest regression model to the training data using the best parameters found in Question 2.c

## Training a Random Forest using the best value for the  n_estimators and max_samples
RandFor_Q2 = RandomForestRegressor(n_estimators=grid_regression_Question3.best_params_["n_estimators"],max_samples=grid_regression_Question3.best_params_["max_samples"], max_features=grid_regression_Question3.best_params_["max_features"])
RandFor_Q2.fit(xwhole_training_setQ2, ywhole_training_setQ2)
RandFor_Q2_pred = RandFor_Q2.predict(xtest_setQ2)

In [ ]:
##fourth point - Compute the RMSE over the test data and report the result 
## import of the sklearn mean_squared_error function
from sklearn.metrics import mean_squared_error

RMSE_RandForQ2 = np.sqrt(mean_squared_error(ytest_setQ2, RandFor_Q2_pred))
print('The RMSE on the test data is :',RMSE_RandForQ2)

The RMSE on the test data is : 0.3097051922475832
